##### **** These pip install need to be adapted to use the appropriate release level. Currently used for testing alpha release of transforms 1.0

In [ ]:
%%capture
!pip install 'data-prep-toolkit-transforms[language]==1.0.0a1'
import pyarrow.parquet as pq
import pandas as pd

configur and run web2parquet

In [ ]:
# Must enable nested asynchronous io in a notebook as the crawler uses coroutine to speed up acquisition and downloads
import nest_asyncio
nest_asyncio.apply()

In [ ]:
%%capture
from dpk_web2parquet.transform import Web2Parquet
Web2Parquet(urls= ['https://arxiv.org/pdf/2408.09869'],
                    depth=2, 
                    downloads=10,
                    folder='files-web2parquet').transform()


In [ ]:
##### **** The specified downloads folder will include the downloaded file(s).
#import glob
#glob.glob("files-web2parquet/*") 

Configure and run Pdf2Parquet

In [ ]:
from dpk_pdf2parquet.transform_python import Pdf2Parquet
Pdf2Parquet(input_folder= "files-web2parquet", 
               output_folder= "files-pdf2parquet", 
               data_files_to_use=['.pdf'],
               pdf2parquet_contents_type='text/markdown').transform()

In [ ]:
##### **** To explote the output from pdf2parquet, run the code below
#table = pq.read_table('files-pdf2parquet/arxiv_org_2408.09869v5.pdf_application.parquet')
#table.to_pandas()

Configure and Run DocChunk

In [ ]:
%%capture
from dpk_doc_chunk.transform_python import DocChunk
DocChunk(input_folder='files-pdf2parquet',
        output_folder='files-doc-chunk',
        doc_chunk_chunking_type= "li_markdown").transform()

In [ ]:
##### **** To explote the output from doc-chunk, run the code below
#table = pq.read_table('files-doc-chunk/arxiv_org_2408.09869v5.pdf_application.parquet')
#table.to_pandas()

Configure and Run Exact dedup

In [ ]:
from dpk_ededup.transform_python import Ededup
Ededup(input_folder="files-doc-chunk",
    output_folder="files-ededup",
    ededup_doc_column="contents",
    ededup_doc_id_column="document_id").transform()

In [ ]:
##### **** To explote the output from eDedup, run the code below
#table = pq.read_table('files-ededup/arxiv_org_2408.09869v5.pdf_application.parquet')
#table.to_pandas()

Configure and run Lang Id

In [ ]:
from dpk_lang_id.transform_python import LangId
LangId(input_folder= "files-ededup",
        output_folder= "files-langId",
        lang_id_model_credential= "PUT YOUR OWN HUGGINGFACE CREDENTIAL",
        lang_id_model_kind= "fasttext",
        lang_id_model_url= "facebook/fasttext-language-identification",
        lang_id_content_column_name= "contents").transform()

In [ ]:
##### **** To explote the output from langId, run the code below
#table = pq.read_table('files-langId/arxiv_org_2408.09869v5.pdf_application.parquet')
#table.to_pandas()

Configure and run Doc Quality

In [ ]:
%%capture
from dpk_doc_quality.transform_python import DocQuality
DocQuality(input_folder='files-doc-chunk',
            output_folder= 'files-doc-quality',
            docq_text_lang = "en",
            docq_doc_content_column ="contents").transform()

In [ ]:
##### **** To explote the output from Doc Quality, run the code below
#table = pq.read_table('files-doc-quality/arxiv_org_2408.09869v5.pdf_application.parquet')
#table.to_pandas()

Run tokenization

In [ ]:
%%capture
from dpk_tokenization.transform_python import Tokenization
Tokenization(input_folder= "files-doc-quality",
        output_folder= "files-tokenization",
        tkn_tokenizer=  "hf-internal-testing/llama-tokenizer",
        tkn_chunk_size= 20_000).transform()

In [ ]:
##### **** To explote the output from Doc Quality, run the code below
#table = pq.read_table('files-tokenization/arxiv_org_2408.09869v5.pdf_application.parquet')
#table.to_pandas()

Configure and Run Fuzzy dedup

In [ ]:
from dpk_fdedup.transform_python import Fdedup
Fdedup(input_folder='files-doc-chunk',
    output_folder='files-fdedup',
    contents_column= "contents",
    document_id_column= "document_id",
    num_permutations= 112,
    num_bands= 14,
    num_minhashes_per_band= 8,
    operation_mode="filter_duplicates").transform()

In [ ]:
##### **** To explote the output from eDedup, run the code below
#table = pq.read_table('files-fdedup/arxiv_org_2408.09869v5.pdf_application.parquet')
#table.to_pandas()